# Day 3: Disney Parks Guest Spending Behavior

You are a data analyst working with the Disney Parks revenue team to understand nuanced guest spending patterns across different park experiences. The team wants to develop a comprehensive view of visitor purchasing behaviors. Your goal is to uncover meaningful insights that can drive personalized marketing strategies.

In [ ]:
import pandas as pd
import numpy as np

fct_guest_spending_data = [
  {
    "guest_id": 1,
    "visit_date": "2024-07-05",
    "amount_spent": 50,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-07-06",
    "amount_spent": 30,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-07-10",
    "amount_spent": 20.5,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 4,
    "visit_date": "2024-07-12",
    "amount_spent": 40,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-07-15",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 5,
    "visit_date": "2024-07-20",
    "amount_spent": 60,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 6,
    "visit_date": "2024-07-25",
    "amount_spent": 25,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-03",
    "amount_spent": 55,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-15",
    "amount_spent": 45,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-05",
    "amount_spent": 22,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-20",
    "amount_spent": 38,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 7,
    "visit_date": "2024-08-10",
    "amount_spent": 15,
    "park_experience_type": "Character Meet"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-25",
    "amount_spent": 28,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-27",
    "amount_spent": 32,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-02",
    "amount_spent": 65,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-05",
    "amount_spent": 50,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 9,
    "visit_date": "2024-09-15",
    "amount_spent": 40,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 10,
    "visit_date": "2024-09-20",
    "amount_spent": 70,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-25",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-28",
    "amount_spent": 10,
    "park_experience_type": "Character Meet"
  }
]
fct_guest_spending = pd.DataFrame(fct_guest_spending_data)


## Question 1

What is the average spending per guest per visit for each park experience type during July 2024? Ensure that park experience types with no recorded transactions are shown with an average spending of 0.0. This analysis helps establish baseline spending differences essential for later segmentation.

In [ ]:
startOfMonth = fct_guest_spending['visit_date'] >= '2024-07-01'
endOfMonth = fct_guest_spending['visit_date'] <= '2024-07-31'

julyFilter = fct_guest_spending[startOfMonth & endOfMonth]

average_spending = julyFilter.groupby(['park_experience_type', 'guest_id']).agg(
    total_spent=('amount_spent', 'sum'),
    total_visit=('guest_id', 'count')
).reset_index()

average_spending = average_spending.groupby('park_experience_type').agg(
    total_spent=('total_spent', 'sum'),
    total_visit=('total_visit', 'sum')
).reset_index()

average_spending['average_spending'] = average_spending['total_spent'] / average_spending['total_visit']

average_spending['average_spending'] = average_spending['average_spending'].fillna(0.0)

all_experience_types = pd.DataFrame({
    'park_experience_type': ['Attraction', 'Dining', 'Entertainment', 'Retail']
})

final_result = pd.merge(all_experience_types, average_spending[['park_experience_type', 'average_spending']], on="park_experience_type", how='left')

final_result['average_spending'] = final_result['average_spending'].fillna(0.0)

print(final_result)

## Question 2

For guests who visited our parks more than once in August 2024, what is the difference in spending between their first and their last visit? This investigation, using sequential analysis, will reveal any shifts in guest spending behavior over multiple visits.

In [ ]:
startOfMonth = fct_guest_spending['visit_date'] >= '2024-08-01'
endOfMonth = fct_guest_spending['visit_date'] <= '2024-08-31'
monthFilter = fct_guest_spending[startOfMonth & endOfMonth]

guest_visits = monthFilter.groupby('guest_id').filter(lambda x: len(x) > 1)
guest_visits = guest_visits.sort_values(by=['guest_id', 'visit_date'])
first_last_spending = guest_visits.groupby('guest_id').agg(
    first_visit_spending=('amount_spent', 'first'),
    last_visit_spending=('amount_spent', 'last')
).reset_index()
first_last_spending['spending_difference'] = first_last_spending['last_visit_spending'] - first_last_spending['first_visit_spending']
print(first_last_spending[['guest_id', 'first_visit_spending', 'last_visit_spending', 'spending_difference']])

## Question 3

In September 2024, how can guests be categorized into distinct spending segments such as Low, Medium, and High based on their total spending? Use the following thresholds for categorization: 
-Low: Includes values from $0 up to, but not including, $50.
-Medium: Includes values from $50 up to, but not including, $100.
-High: Includes values from $100 and above. 
Exclude guests who did not make any purchases in the period.

In [ ]:
startOfMonth = fct_guest_spending['visit_date'] >= '2024-09-01'
endOfMonth = fct_guest_spending['visit_date'] <= '2024-09-30'
septemberFilter = fct_guest_spending[startOfMonth & endOfMonth]

total_spending = septemberFilter.groupby('guest_id').agg(
    total_spent=('amount_spent', 'sum')
).reset_index()

total_spending = total_spending[total_spending['total_spent'] > 0]

def categorize_spending(spending):
    if spending < 50:
        return 'Low'
    elif spending < 100:
        return 'Medium'
    else:
        return 'High'

total_spending['spending_segment'] = total_spending['total_spent'].apply(categorize_spending)

print(total_spending[['guest_id', 'total_spent', 'spending_segment']])

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)